In [175]:
import pandas as pd
import numpy as np
import requests
import datetime
import os
sys.path.append('../..')

In [176]:
from data.dataloader import get_covid19india_api_data, get_rootnet_api_data

In [177]:
dataframes = get_covid19india_api_data()

In [178]:
dataframes.keys()

dict_keys(['df_tested', 'df_statewise', 'df_india_time_series', 'df_districtwise', 'df_raw_data', 'df_raw_data_2', 'df_travel_history', 'df_resources'])

In [286]:

districtwise = dataframes['df_districtwise']
districtwise[districtwise['state'] == 'Gujarat']['district'].unique()
# districtwise[districtwise['state'] == 'Gujarat']

array(['Ahmadabad', 'Anand', 'Aravalli', 'Banas Kantha', 'Bharuch',
       'Bhavnagar', 'Botad', 'Chota Udaipur', 'Dahod', 'Gandhinagar',
       'Gir Somnath', 'Jamnagar', 'Kachchh', 'Kheda', 'Mahesana',
       'Mahisagar', 'Morbi', 'Narmada', 'Navsari', 'Panch Mahals',
       'Patan', 'Porbandar', 'Rajkot', 'Sabar Kantha', 'Surat', 'Tapi',
       'Dang', 'Vadodara', 'Valsad', 'Unknown'], dtype=object)

In [366]:
dataframes['df_raw_data_2'].columns
xyz = dataframes['df_raw_data_2']
deceased = xyz[xyz['patientstatus'] == 'Deceased']
# deceased[deceased['district'] == '']['state'].value_counts()
state = deceased[deceased['state'] == 'Maharashtra']
#  MANUAL CLEANUP + ASSUMPTIONS MADE IN CODE AFTER PRELIM MANUAL REVIEW
state[state['district'] == ''].index[:20]
state.loc[state[state['district'] == ''].index[:20]]
# gj[gj['district'] == '']['district'] = 'Ahmadabad'


,agebracket,city,date,district,gender,nationality,notes,patientnumbercouldbemappedlater,patientstatus,slno,source1,source2,source3,state,statecode
235,,,02/04/2020,,,,Details awaited,,Deceased,248,https://twitter.com/ANI/status/124569425319748...,,,Maharashtra,MH
236,,,02/04/2020,,,,Details awaited,,Deceased,249,https://twitter.com/ANI/status/124569425319748...,,,Maharashtra,MH
237,,,02/04/2020,,,,Details awaited,,Deceased,250,https://twitter.com/ANI/status/124569425319748...,,,Maharashtra,MH
239,,,02/04/2020,,,,Details awaited,,Deceased,262,https://twitter.com/PTI_News/status/1245752854...,,,Maharashtra,MH
269,,,03/04/2020,,,,Details awaited,,Deceased,302,https://twitter.com/PTI_News/status/1246098331...,,,Maharashtra,MH
270,,,03/04/2020,,,,Details awaited,,Deceased,303,https://twitter.com/PTI_News/status/1246098331...,,,Maharashtra,MH
271,,,03/04/2020,,,,Details awaited,,Deceased,304,https://twitter.com/PTI_News/status/1246098331...,,,Maharashtra,MH
272,,,03/04/2020,,,,Details awaited,,Deceased,305,https://twitter.com/PTI_News/status/1246098331...,,,Maharashtra,MH
273,,,03/04/2020,,,,Details awaited,,Deceased,306,https://twitter.com/PTI_News/status/1246098331...,,,Maharashtra,MH
778,,,09/04/2020,,,,Details awaited,,Deceased,815,https://arogya.maharashtra.gov.in/pdf/covidmpr...,,,Maharashtra,MH


In [367]:
def get_district_death_df(df_raw_data_2, state, district):
    deceased = df_raw_data_2[df_raw_data_2['patientstatus'] == 'Deceased']
    statedf = deceased[deceased['state'] == state]
    unknown = statedf[statedf['district'] == '']
    print(f'{len(unknown)} deaths in {state} with unknown district')
    if district == 'Ahmadabad':
        print(f'adding {len(unknown)} deaths to Ahmadabad count')
        statedf['district'][statedf['district'] == ''] = district
        districtdf = statedf[statedf['district'] == district]
    elif state == 'Maharashtra':
        num_unknown = len(unknown)
        thirds = num_unknown//3
        print(f'adding {thirds} deaths to Mumbai/Pune each')
        print(len(statedf[statedf['district'] == district]))
        unknown_index = statedf['district'][statedf['district'] == ''].index
        statedf.loc[statedf[statedf['district'] == ''].index[:thirds]] = 'Mumbai'
        statedf.loc[statedf[statedf['district'] == ''].index[thirds:]] = 'Pune'
        print(len(statedf[statedf['district'] == district]))
        districtdf = statedf[statedf['district'] == district]
    elif state == 'Rajasthan':
        districtdf = statedf[statedf['district'] == district]
    elif state == 'Karnataka':
        districtdf = statedf[statedf['district'] == district]
    elif state == 'Delhi':
        districtdf = statedf[statedf['district'] == district]
    return districtdf

In [368]:

def get_district_time_series(dataframes, state='Karnataka', district='Bengaluru'):
    if district == 'all' or type(district) == list:
        if district == 'all':
            districtwise = dataframes['df_districtwise']
            district = districtwise[districtwise['state'] == state]['district'].unique()
            state = len(district) * [state]
        district_timeseries = {}
        for (s, d) in list(zip(state, districts)):
            district_timeseries[d] = get_district_time_series(dataframes, state=s, district=d)
        return district_timeseries
    else:    
        df_raw_data_1 = dataframes['df_raw_data'][dataframes['df_raw_data']['detectedstate'] == state]
        df_raw_data_1 = df_raw_data_1[df_raw_data_1['detecteddistrict'] == district]
        df_raw_data_1['dateannounced'] = pd.to_datetime(df_raw_data_1['dateannounced'], format='%d/%m/%Y')

        index = pd.date_range(np.min(df_raw_data_1['dateannounced']), np.max(df_raw_data_1['dateannounced']))

        df_district = pd.DataFrame(columns=['total_confirmed'], index=index)
        df_district['total_confirmed'] = [0]*len(index)
        for _, row in df_raw_data_1.iterrows():
            df_district.loc[row['dateannounced']:, 'total_confirmed'] += 1

        # Deaths calculation
        deathsdf = get_district_death_df(dataframes['df_raw_data_2'], state, district)
        deathsdf = deathsdf[deathsdf['state'] == state]
        deathsdf = deathsdf[deathsdf['district'] == district]
        deathsdf['date'] = pd.to_datetime(deathsdf['date'], format='%d/%m/%Y')

        df_district['total_deaths'] = [0]*len(index)
        for _, row in deathsdf.iterrows():
            if row['patientstatus'] == 'Deceased':
                date = pd.to_datetime(row['date'], format='%d/%m/%Y')
                df_district.loc[date:, 'total_deaths'] += 1


        df_district.reset_index(inplace=True)
        df_district.columns = ['date', 'total_confirmed', 'total_deaths']
        return df_district


In [369]:
districts = ['Mumbai', 'Bengaluru', 'Ahmadabad', 'Jaipur', 'Pune', 'New Delhi']
district_timeseries = get_district_time_series(dataframes, state=['Maharashtra', 'Karnataka', 'Gujarat', 'Rajasthan', 'Maharashtra', 'Delhi'], district=districts)
district_timeseries.keys()

60 deaths in Maharashtra with unknown district
adding 20 deaths to Mumbai/Pune each
153
173
4 deaths in Karnataka with unknown district
21 deaths in Gujarat with unknown district
adding 21 deaths to Ahmadabad count
19 deaths in Rajasthan with unknown district
60 deaths in Maharashtra with unknown district
adding 20 deaths to Mumbai/Pune each
46
66
50 deaths in Delhi with unknown district


dict_keys(['Mumbai', 'Bengaluru', 'Ahmadabad', 'Jaipur', 'Pune', 'New Delhi'])

In [388]:
data = district_timeseries['Ahmadabad'].set_index('date')
data[:5]

,total_confirmed,total_deaths
date,,
2020-03-20,3,0
2020-03-21,5,0
2020-03-22,8,0
2020-03-23,14,0
2020-03-24,14,0


In [389]:
# get age data in bands
age_data = {}
directory = '../../data/data/census/'
for filename in os.listdir(directory):
    df = pd.read_excel(os.path.join(directory, filename))
    age_data[filename.split('.')[0]] = df.dropna(how='all')

age_data.keys()

dict_keys(['DDW-2400C-13', 'DDW-2900C-13', 'DDW-0800C-13', 'DDW-2700C-13', 'DDW-0700C-13'])

In [390]:
raw_all_district_age_data = age_data['DDW-2400C-13']


In [391]:
def clean(raw_all_district_age_data):
    transposed = raw_all_district_age_data.head(3).T
    transposed.fillna('', inplace=True)
    new = transposed[transposed.columns[0]]
    for x in transposed.columns[1:]:
        print (transposed[x])
        new += transposed[x] 
    all_district_age_data = raw_all_district_age_data.copy()
    all_district_age_data.columns = new.T
    return all_district_age_data[4:]

In [392]:
all_district_age_data = clean(raw_all_district_age_data)


Unnamed: 0                                            Name
Unnamed: 1                                            Code
Unnamed: 2                                            Code
Unnamed: 3                                                
C-13 SINGLE YEAR AGE RETURNS BY RESIDENCE AND SEX         
Unnamed: 5                                                
Unnamed: 6                                                
Unnamed: 7                                                
Unnamed: 8                                                
Unnamed: 9                                                
Unnamed: 10                                               
Unnamed: 11                                               
Unnamed: 12                                               
Unnamed: 13                                               
Name: 1, dtype: object
Unnamed: 0                                                   
Unnamed: 1                                                   
Unnamed: 2                 

In [393]:
# Get relevant district(s) data
district_age_data = all_district_age_data[all_district_age_data['Area Name'] == 'District - Ahmadabad (07)']
# or district_age_data['Distt.Code'] = 474 

In [394]:
district_age_data['Age'].unique()

array(['All ages', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,
       84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
       '100+', 'Age not stated'], dtype=object)

In [395]:
age_bands = {}
for a in range(9):
    lower = a * 10
    upper = lower + 9
    if lower == 80:
        age_bands[f'{lower}+'] = list(range(lower, upper+11))
        age_bands[f'{lower}+'] += ['100+']
    else:
        age_bands[f'{lower}-{upper}'] = list(range(lower, upper+1))
    
age_bands

{'0-9': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 '10-19': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 '20-29': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 '30-39': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 '40-49': [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 '50-59': [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 '60-69': [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 '70-79': [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
 '80+': [80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  '100+']}

In [396]:

# filter for age bands and sum
total_pop = district_age_data[district_age_data['Age'] == 'All ages']['TotalPersons'].values[0]
unknown_pop = district_age_data[district_age_data['Age'] == 'Age not stated']['TotalPersons'].values[0]
total_pop -= unknown_pop

age_band_pops = {}
for key, band in age_bands.items():
    pop = district_age_data[district_age_data['Age'].isin(band)]['TotalPersons'].sum()
    age_band_pops[key] = pop
age_band_pops

{'0-9': 1216635,
 '10-19': 1348459,
 '20-29': 1387764,
 '30-39': 1144310,
 '40-49': 899585,
 '50-59': 620550,
 '60-69': 354147,
 '70-79': 158438,
 '80+': 65017}

In [397]:
total_pop == sum(age_band_pops.values())

True

In [398]:
age_band_ratios = {k: v / total_pop for k, v in age_band_pops.items()}
age_band_ratios

{'0-9': 0.16909674276449793,
 '10-19': 0.18741859690989665,
 '20-29': 0.19288149044358474,
 '30-39': 0.15904449051099354,
 '40-49': 0.12503083779424468,
 '50-59': 0.08624853281593016,
 '60-69': 0.04922191467434247,
 '70-79': 0.022020860595101673,
 '80+': 0.009036533491408157}

In [399]:
# https://www.thelancet.com/action/showPdf?pii=S1473-3099%2820%2930243-7
# this paper has china-based age-stratified case fatality, infection fatality, and infected-hospitalization ratios
# more deaths were reported afterwards, though, so not the best source

# for now, going to use "Adjusted for censoring, demography, and underascertainment‡" column
ref_mortality_rate = {
    '0-9':   0.00,
    '10-19': 0.01,
    '20-29': 0.06,
    '30-39': 0.15,
    '40-49': 0.30,
    '50-59': 1.25,
    '60-69': 3.99,
    '70-79': 8.61,
    '80+': 13.40,
}

In [400]:
implied_mortality_age = {}
for k in ref_mortality_rate.keys():
    implied_mortality_age[k] = ref_mortality_rate[k] * age_band_ratios[k]

implied_mortality = sum(implied_mortality_age.values())

In [401]:
# need mortality rate from India -- this is going to be dependent on case data, so introducing that testing bias...
daily_observed_mortality = data['total_deaths']/data['total_confirmed']
daily_observed_mortality

date
2020-03-20    0.000000
2020-03-21    0.000000
2020-03-22    0.000000
2020-03-23    0.000000
2020-03-24    0.000000
2020-03-25    0.000000
2020-03-26    0.000000
2020-03-27    0.000000
2020-03-28    0.000000
2020-03-29    0.142857
2020-03-30    0.136364
2020-03-31    0.130435
2020-04-01    0.096774
2020-04-02    0.096774
2020-04-03    0.105263
2020-04-04    0.111111
2020-04-05    0.094340
2020-04-06    0.078125
2020-04-07    0.060241
2020-04-08    0.060241
2020-04-09    0.049296
2020-04-10    0.040609
2020-04-11    0.045267
2020-04-12    0.046099
2020-04-13    0.043750
2020-04-14    0.037534
2020-04-15    0.037778
2020-04-16    0.033028
2020-04-17    0.035313
2020-04-18    0.030162
2020-04-19    0.029973
2020-04-20    0.031250
2020-04-21    0.039330
2020-04-22    0.041972
2020-04-23    0.042373
dtype: float64

In [402]:
daily_mortality_ratio = daily_observed_mortality/implied_mortality
daily_mortality_ratio

date
2020-03-20    0.000000
2020-03-21    0.000000
2020-03-22    0.000000
2020-03-23    0.000000
2020-03-24    0.000000
2020-03-25    0.000000
2020-03-26    0.000000
2020-03-27    0.000000
2020-03-28    0.000000
2020-03-29    0.207127
2020-03-30    0.197712
2020-03-31    0.189116
2020-04-01    0.140312
2020-04-02    0.140312
2020-04-03    0.152620
2020-04-04    0.161099
2020-04-05    0.136782
2020-04-06    0.113273
2020-04-07    0.087343
2020-04-08    0.087343
2020-04-09    0.071473
2020-04-10    0.058879
2020-04-11    0.065633
2020-04-12    0.066839
2020-04-13    0.063433
2020-04-14    0.054419
2020-04-15    0.054774
2020-04-16    0.047886
2020-04-17    0.051200
2020-04-18    0.043732
2020-04-19    0.043457
2020-04-20    0.045309
2020-04-21    0.057024
2020-04-22    0.060855
2020-04-23    0.061436
dtype: float64

In [403]:
age_stratified_daily_mortality = {}
for k in ref_mortality_rate.keys():
    age_stratified_daily_mortality[k] = daily_mortality_ratio * ref_mortality_rate[k]

In [404]:
age_distro_mortality = {}
for k in age_band_ratios.keys():
    age_distro_mortality[k] = age_stratified_daily_mortality[k] * age_band_ratios[k]


In [405]:
age_distro_mortality_vals = list(age_distro_mortality.values())
age_std_mortality = age_distro_mortality_vals[0]
for val in age_distro_mortality_vals[1:]:
    age_std_mortality += val
age_std_mortality

date
2020-03-20    0.000000
2020-03-21    0.000000
2020-03-22    0.000000
2020-03-23    0.000000
2020-03-24    0.000000
2020-03-25    0.000000
2020-03-26    0.000000
2020-03-27    0.000000
2020-03-28    0.000000
2020-03-29    0.142857
2020-03-30    0.136364
2020-03-31    0.130435
2020-04-01    0.096774
2020-04-02    0.096774
2020-04-03    0.105263
2020-04-04    0.111111
2020-04-05    0.094340
2020-04-06    0.078125
2020-04-07    0.060241
2020-04-08    0.060241
2020-04-09    0.049296
2020-04-10    0.040609
2020-04-11    0.045267
2020-04-12    0.046099
2020-04-13    0.043750
2020-04-14    0.037534
2020-04-15    0.037778
2020-04-16    0.033028
2020-04-17    0.035313
2020-04-18    0.030162
2020-04-19    0.029973
2020-04-20    0.031250
2020-04-21    0.039330
2020-04-22    0.041972
2020-04-23    0.042373
dtype: float64

In [406]:
log_age_std_mortality = np.log(age_std_mortality)
log_age_std_mortality

date
2020-03-20        -inf
2020-03-21        -inf
2020-03-22        -inf
2020-03-23        -inf
2020-03-24        -inf
2020-03-25        -inf
2020-03-26        -inf
2020-03-27        -inf
2020-03-28        -inf
2020-03-29   -1.945910
2020-03-30   -1.992430
2020-03-31   -2.036882
2020-04-01   -2.335375
2020-04-02   -2.335375
2020-04-03   -2.251292
2020-04-04   -2.197225
2020-04-05   -2.360854
2020-04-06   -2.549445
2020-04-07   -2.809403
2020-04-08   -2.809403
2020-04-09   -3.009917
2020-04-10   -3.203762
2020-04-11   -3.095166
2020-04-12   -3.076958
2020-04-13   -3.129264
2020-04-14   -3.282521
2020-04-15   -3.276034
2020-04-16   -3.410414
2020-04-17   -3.343504
2020-04-18   -3.501159
2020-04-19   -3.507467
2020-04-20   -3.465736
2020-04-21   -3.235769
2020-04-22   -3.170752
2020-04-23   -3.161247
dtype: float64